In [5]:
import pandas as pd, plotly as ply, numpy as np, os

In [10]:
from datasets import load_dataset, load_dataset_builder

In [13]:
ds_builder = load_dataset_builder("openai/summarize_from_feedback", "comparisons")

In [14]:
ds_builder.info.description

'Summarize from Feedback contains the human feedback data released by the "Learning to summarize from human feedback" paper.\n'

In [15]:
ds_builder.info.features

{'info': {'id': Value(dtype='string', id=None),
  'post': Value(dtype='string', id=None),
  'title': Value(dtype='string', id=None),
  'subreddit': Value(dtype='string', id=None),
  'site': Value(dtype='string', id=None),
  'article': Value(dtype='string', id=None)},
 'summaries': [{'text': Value(dtype='string', id=None),
   'policy': Value(dtype='string', id=None),
   'note': Value(dtype='string', id=None)}],
 'choice': Value(dtype='int32', id=None),
 'worker': Value(dtype='string', id=None),
 'batch': Value(dtype='string', id=None),
 'split': Value(dtype='string', id=None),
 'extra': {'confidence': Value(dtype='int32', id=None)}}

In [16]:
train_data = load_dataset('openai/summarize_from_feedback', "comparisons", split ='train[0:500]')

Extracting data files:   0%|          | 0/23 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset summarize_from_feedback downloaded and prepared to /Users/johnwang/.cache/huggingface/datasets/openai___summarize_from_feedback/comparisons/0.0.0/483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0. Subsequent calls will reuse this data.


In [17]:
train_data

Dataset({
    features: ['info', 'summaries', 'choice', 'worker', 'batch', 'split', 'extra'],
    num_rows: 500
})

In [33]:
df = train_data.to_pandas()

In [34]:
df.head()

,info,summaries,choice,worker,batch,split,extra
0,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' Mum is mad at me for not flying on...,1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,{'confidence': None}
1,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' I have made sure my mother is comf...,1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,{'confidence': None}
2,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' mum isn't speaking to me because I...,0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,{'confidence': None}
3,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' Mum thought I was going to road tr...,0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,{'confidence': None}
4,"{'id': 't3_1zwek5', 'post': 'My landlord left ...",[{'text': ' My landlord is harassing me and my...,1,LjvoXOAj5op3WqNnn5b7TZTG8mK7gM,batch3,train,{'confidence': None}


In [35]:
df[df['split']=='train'].shape

(500, 7)

In [36]:
df = df.drop(["batch", "split", "extra", "worker"], axis="columns")

In [37]:
df.head()

,info,summaries,choice
0,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' Mum is mad at me for not flying on...,1
1,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' I have made sure my mother is comf...,1
2,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' mum isn't speaking to me because I...,0
3,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' Mum thought I was going to road tr...,0
4,"{'id': 't3_1zwek5', 'post': 'My landlord left ...",[{'text': ' My landlord is harassing me and my...,1


In [38]:
df["summaries"][0]

array([{'text': ' Mum is mad at me for not flying on my own trip to meet my boyfriend.', 'policy': 'sup1', 'note': None},
       {'text': ' I have made sure my mother is comfortable with my boyfriend travelling on a trip and now my mother is mad because I booked it.', 'policy': 'sup1', 'note': None}],
      dtype=object)

In [46]:
df['post'] = df['info'].str['post']
df['title'] = df['info'].str['title']
df['subreddit'] = df['info'].str['subreddit']

In [47]:
df

,info,summaries,choice,prompt,post,title,subreddit
0,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' Mum is mad at me for not flying on...,1,None,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships
1,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' I have made sure my mother is comf...,1,None,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships
2,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' mum isn't speaking to me because I...,0,None,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships
3,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' Mum thought I was going to road tr...,0,None,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships
4,"{'id': 't3_1zwek5', 'post': 'My landlord left ...",[{'text': ' My landlord is harassing me and my...,1,None,My landlord left a falsified message taped to ...,Can I sue my property management company and l...,AskReddit
...,...,...,...,...,...,...,...
495,"{'id': 't3_27uaof', 'post': 'Yo, so me and my ...",[{'text': ' My friend and I talked for a few h...,0,None,"Yo, so me and my friend were texting for LITER...",Me [16 M] with my friend [16 F] Just got cold-...,relationships
496,"{'id': 't3_27uaof', 'post': 'Yo, so me and my ...",[{'text': ' Texted my friend for 7 hours a nig...,0,None,"Yo, so me and my friend were texting for LITER...",Me [16 M] with my friend [16 F] Just got cold-...,relationships
497,"{'id': 't3_27uaof', 'post': 'Yo, so me and my ...",[{'text': ' I texted my girl for 8 hours and n...,1,None,"Yo, so me and my friend were texting for LITER...",Me [16 M] with my friend [16 F] Just got cold-...,relationships
498,"{'id': 't3_27uaof', 'post': 'Yo, so me and my ...","[{'text': ' I have been texting for 8 hours, b...",0,None,"Yo, so me and my friend were texting for LITER...",Me [16 M] with my friend [16 F] Just got cold-...,relationships


In [51]:
df = df.drop("info", axis="columns")
df["question_id"] = pd.Series(range(df.shape[0]))
df["category"] = pd.Series((["summary"]*df.shape[0]))
df

,summaries,choice,prompt,post,title,subreddit,question_id,category
0,[{'text': ' Mum is mad at me for not flying on...,1,None,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,0,summary
1,[{'text': ' I have made sure my mother is comf...,1,None,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,1,summary
2,[{'text': ' mum isn't speaking to me because I...,0,None,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,2,summary
3,[{'text': ' Mum thought I was going to road tr...,0,None,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,3,summary
4,[{'text': ' My landlord is harassing me and my...,1,None,My landlord left a falsified message taped to ...,Can I sue my property management company and l...,AskReddit,4,summary
...,...,...,...,...,...,...,...,...
495,[{'text': ' My friend and I talked for a few h...,0,None,"Yo, so me and my friend were texting for LITER...",Me [16 M] with my friend [16 F] Just got cold-...,relationships,495,summary
496,[{'text': ' Texted my friend for 7 hours a nig...,0,None,"Yo, so me and my friend were texting for LITER...",Me [16 M] with my friend [16 F] Just got cold-...,relationships,496,summary
497,[{'text': ' I texted my girl for 8 hours and n...,1,None,"Yo, so me and my friend were texting for LITER...",Me [16 M] with my friend [16 F] Just got cold-...,relationships,497,summary
498,"[{'text': ' I have been texting for 8 hours, b...",0,None,"Yo, so me and my friend were texting for LITER...",Me [16 M] with my friend [16 F] Just got cold-...,relationships,498,summary
